In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Flatten, Dense, Input, concatenate
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dropout
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.optimizers import SGD, Adam
import matplotlib .pyplot as plt

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.metrics import accuracy_score

from datetime import datetime
import cv2
import zipfile
import shutil
import os
import glob

from tensorflow.keras.applications import Xception, ResNet50
from tensorflow.keras.applications import InceptionV3, MobileNet

In [41]:
df = pd.read_csv("project/tomato_final_df.csv")
df

,imagePath,imageHeight,imageWidth,fileVersion,farmId,week,number,type,flowerCluster,date,...,numberOfLeaves,leafLength,leafWidth,floweringNode,numberOfTheFlower,numberOfTheFlowerPerTruss,numberOfFruitPerPlant,numberOfFruitPerTruss,fruitingNode,numberOfTheFlowerFullyDeveloped
0,V001_tom1_39_001_a1_00_20210930_14_00134902_49...,1920,1080,V001,tom1,39,1,a1,0,20210930,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,V001_tom1_39_006_a1_00_20210930_14_00130645_49...,1920,1080,V001,tom1,39,6,a1,0,20210930,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,V001_tom1_39_010_a1_00_20210930_14_00124109_49...,1920,1080,V001,tom1,39,10,a1,0,20210930,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,V001_tom1_39_011_a1_00_20210930_14_00122602_49...,1920,1080,V001,tom1,39,11,a1,0,20210930,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,V001_tom1_39_011_a1_00_20210930_14_00124155_49...,1920,1080,V001,tom1,39,11,a1,0,20210930,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146365,V001_tom6_48_026_f_09_20211202_13_00100240_376...,1920,1080,V001,tom6,48,26,f,9,20211202,...,NaN,NaN,NaN,9.0,1.0,9.0,NaN,NaN,NaN,1.0
146366,V001_tom6_48_026_f_09_20211202_13_01100240_376...,1920,1080,V001,tom6,48,26,f,9,20211202,...,NaN,NaN,NaN,9.0,1.0,9.0,NaN,NaN,NaN,1.0
146367,V001_tom6_48_027_f_07_20211202_13_00144857_376...,1920,1080,V001,tom6,48,27,f,7,20211202,...,NaN,NaN,NaN,7.0,1.0,7.0,NaN,NaN,NaN,1.0
146368,V001_tom6_48_027_f_07_20211202_13_01144857_376...,1920,1080,V001,tom6,48,27,f,7,20211202,...,NaN,NaN,NaN,7.0,1.0,7.0,NaN,NaN,NaN,1.0


In [42]:
df1 = df[df["imagePath"].str.contains("tom1")]
df1 = df1.fillna(0)

In [43]:
df1["date"] = pd.to_datetime(df1["date"], format="%Y%m%d")
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29702 entries, 0 to 145074
Data columns (total 24 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   imagePath                        29702 non-null  object        
 1   imageHeight                      29702 non-null  int64         
 2   imageWidth                       29702 non-null  int64         
 3   fileVersion                      29702 non-null  object        
 4   farmId                           29702 non-null  object        
 5   week                             29702 non-null  int64         
 6   number                           29702 non-null  int64         
 7   type                             29702 non-null  object        
 8   flowerCluster                    29702 non-null  int64         
 9   date                             29702 non-null  datetime64[ns]
 10  plantHeight                      29702 non-null  float64 

In [44]:
df1.describe()

,imageHeight,imageWidth,week,number,flowerCluster,plantHeight,weeklyGrowth,heightOfTheFlowerTruss,stemDiameter,numberOfLeaves,leafLength,leafWidth,floweringNode,numberOfTheFlower,numberOfTheFlowerPerTruss,numberOfFruitPerPlant,numberOfFruitPerTruss,fruitingNode,numberOfTheFlowerFullyDeveloped
count,29702.0,29702.0,29702.000000,29702.000000,29702.000000,29702.000000,29702.000000,29702.000000,29702.000000,29702.000000,29702.000000,29702.000000,29702.000000,29702.000000,29702.000000,29702.000000,29702.000000,29702.000000,29702.000000
mean,1920.0,1080.0,43.981348,49.478419,8.409535,2.331095,2.331095,0.902922,0.064671,0.715171,0.039868,0.028015,2.004242,0.476029,2.004242,1.249411,5.874083,5.874083,0.027608
std,0.0,0.0,6.879537,27.615897,4.923571,7.674465,7.674465,5.100073,0.291506,3.606482,1.431209,1.004303,4.618322,1.054816,4.618322,1.275445,5.330007,5.330007,0.163848
min,1920.0,1080.0,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1920.0,1080.0,42.000000,26.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1920.0,1080.0,44.000000,50.000000,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,7.000000,7.000000,0.000000
75%,1920.0,1080.0,48.000000,72.000000,12.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,10.000000,10.000000,0.000000
max,1920.0,1080.0,52.000000,100.000000,45.000000,52.460000,52.460000,74.900000,2.580000,21.000000,74.800000,48.500000,18.000000,7.000000,18.000000,6.000000,18.000000,18.000000,1.000000


In [45]:
df1.columns

Index(['imagePath', 'imageHeight', 'imageWidth', 'fileVersion', 'farmId',
       'week', 'number', 'type', 'flowerCluster', 'date', 'plantHeight',
       'weeklyGrowth', 'heightOfTheFlowerTruss', 'stemDiameter',
       'numberOfLeaves', 'leafLength', 'leafWidth', 'floweringNode',
       'numberOfTheFlower', 'numberOfTheFlowerPerTruss',
       'numberOfFruitPerPlant', 'numberOfFruitPerTruss', 'fruitingNode',
       'numberOfTheFlowerFullyDeveloped'],
      dtype='object')

In [46]:
df2 = df1[['imagePath','date', 'plantHeight', 'weeklyGrowth', 'heightOfTheFlowerTruss', 'stemDiameter',
       'numberOfLeaves', 'leafLength', 'leafWidth', 'floweringNode',
       'numberOfTheFlower', 'numberOfTheFlowerPerTruss',
       'numberOfFruitPerPlant', 'numberOfFruitPerTruss', 'fruitingNode',
       'numberOfTheFlowerFullyDeveloped']]
df2

,imagePath,date,plantHeight,weeklyGrowth,heightOfTheFlowerTruss,stemDiameter,numberOfLeaves,leafLength,leafWidth,floweringNode,numberOfTheFlower,numberOfTheFlowerPerTruss,numberOfFruitPerPlant,numberOfFruitPerTruss,fruitingNode,numberOfTheFlowerFullyDeveloped
0,V001_tom1_39_001_a1_00_20210930_14_00134902_49...,2021-09-30,37.10,37.10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,V001_tom1_39_006_a1_00_20210930_14_00130645_49...,2021-09-30,35.15,35.15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,V001_tom1_39_010_a1_00_20210930_14_00124109_49...,2021-09-30,38.69,38.69,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,V001_tom1_39_011_a1_00_20210930_14_00122602_49...,2021-09-30,43.58,43.58,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,V001_tom1_39_011_a1_00_20210930_14_00124155_49...,2021-09-30,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145070,V001_tom1_48_034_f_14_20211202_23_00110435_401...,2021-12-02,0.00,0.00,0.0,0.0,0.0,0.0,0.0,14.0,1.0,14.0,0.0,0.0,0.0,1.0
145071,V001_tom1_48_037_f_15_20211202_23_00111218_401...,2021-12-02,0.00,0.00,0.0,0.0,0.0,0.0,0.0,15.0,1.0,15.0,0.0,0.0,0.0,1.0
145072,V001_tom1_48_069_f_14_20211204_23_00135923_381...,2021-12-04,0.00,0.00,0.0,0.0,0.0,0.0,0.0,14.0,1.0,14.0,0.0,0.0,0.0,1.0
145073,V001_tom1_50_031_f_16_20211216_25_00105930_401...,2021-12-16,0.00,0.00,0.0,0.0,0.0,0.0,0.0,16.0,1.0,16.0,0.0,0.0,0.0,1.0


In [47]:
cols = ['파일이름','날짜', '초장', '주간 생장길이', '화방 높이', '줄기 직경',
       '엽수', '엽길이', '엽넓이', '개화 마디', '꽃 수', '꽃 개화군', '착과수', '착과군', '착과 마디',
       '만개 꽃 수']
df2.columns = cols
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29702 entries, 0 to 145074
Data columns (total 16 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   파일이름     29702 non-null  object        
 1   날짜       29702 non-null  datetime64[ns]
 2   초장       29702 non-null  float64       
 3   주간 생장길이  29702 non-null  float64       
 4   화방 높이    29702 non-null  float64       
 5   줄기 직경    29702 non-null  float64       
 6   엽수       29702 non-null  float64       
 7   엽길이      29702 non-null  float64       
 8   엽넓이      29702 non-null  float64       
 9   개화 마디    29702 non-null  float64       
 10  꽃 수      29702 non-null  float64       
 11  꽃 개화군    29702 non-null  float64       
 12  착과수      29702 non-null  float64       
 13  착과군      29702 non-null  float64       
 14  착과 마디    29702 non-null  float64       
 15  만개 꽃 수   29702 non-null  float64       
dtypes: datetime64[ns](1), float64(14), object(1)
memory usage: 3.9+ MB


In [52]:
for i in cols:
    print(df2[i].value_counts())
# 그나마 [엽넓이, 꽃 수, 꽃 개화군, 착과수, 착과군 ]이 데이터수가 어느정도 있음

V001_tom1_39_001_a1_00_20210930_14_00134902_49122255.png    1
V001_tom1_48_052_c_13_20211204_23_03095357_40158887.png     1
V001_tom1_48_053_c_12_20211204_23_00095533_40158887.png     1
V001_tom1_48_053_c_11_20211204_23_01094150_40158887.png     1
V001_tom1_48_053_c_11_20211204_23_00094150_40158887.png     1
                                                           ..
V001_tom1_39_023_c_07_20210930_14_00102020_49122255.png     1
V001_tom1_39_023_c_04_20210930_14_00080836_49122255.png     1
V001_tom1_39_022_c_06_20210930_14_00102436_49122255.png     1
V001_tom1_39_021_c_06_20210930_14_00103805_49122255.png     1
V001_tom1_50_080_f_15_20211218_25_00113036_38102607.png     1
Name: 파일이름, Length: 29702, dtype: int64
2021-11-13    935
2021-10-09    850
2021-11-07    797
2021-12-02    785
2021-10-14    673
             ... 
2021-12-19    129
2022-01-02    120
2021-09-08    110
2021-09-09     59
2021-09-07     51
Name: 날짜, Length: 71, dtype: int64
0.00     26922
21.49       12
25.42       12
